# Build an advanced RAG router based assistant with Amazon Bedrock

[Amazon Bedrock](https://aws.amazon.com/bedrock/) is a fully managed service that offers a choice of high-performing Foundation Models (FMs) from leading AI companies accessible through a single API, along with a broad set of capabilities you need to build generative AI applications, simplifying development while maintaining privacy and security.

[Large Language Models (LLMs)](https://en.wikipedia.org/wiki/Large_language_model) are a type of Foundation Model that can take natural langauge as input, with the ability to process and understand it, and produce natural language as the output. LLMs can also can perform tasks like classification, summarization, simplification, entity recognition, etc.

LLMs are usually trained offline with data that is available until that point of time. As a result, LLMs will not have knowledge of the world after that date. Additionally, LLMs are trained on very general domain corpora, making them less effective for domain-specific tasks. And then, LLMs have the tendency to hallucinate where the model generates text that is incorrect, nonsensical, or not real. Using a [Retrieval Augment Generation (RAG)](https://docs.aws.amazon.com/sagemaker/latest/dg/jumpstart-foundation-models-customize-rag.html) mechanism can help mitigate all these issues. A RAG architecture involves retrieving data that closely matches the text in the user's prompt, from an external datasource, and using it to augment the prompt before sending to the LLM. This prompt augmentation will provide the context that the LLM can use to respond to the prompt.

When there are mulitple datasources, there is a need to route the retreival request to the appropriate datasource before performing the actual retrieval. This is a RAG router pattern.

This notebook will walk you through the process of building an advanced RAG router based assistant using a Large Language Model (LLM) hosted on [Amazon Bedrock](https://aws.amazon.com/bedrock/) and using [Knowledge Bases for Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base.html) for vectorizing, storing, and retrieving data through semantic search. [Amazon OpenSearch Serverless](https://aws.amazon.com/opensearch-service/features/serverless/) will be used as the vector index.

We will use [LangChain](https://www.langchain.com/) to simplify the process of constructing the prompts, and interacting with the LLMs and Knowledge Bases (KBs). In the process of working through this notebook, you will learn how to setup the Amazon Bedrock client environment, configure security permissions and use prompt templates in LangChain. Invocations that involve LangChain will be explicitly mentioned.

![](./images/flowchart.png)

<div class="alert alert-block alert-info">
<b>Note:</b>
    <ul>
        <li>This notebook should only be run from within an <a href="https://docs.aws.amazon.com/sagemaker/latest/dg/nbi.html">Amazon SageMaker Notebook instance</a> or within an <a href="https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated.html">Amazon SageMaker Studio Notebook</a>.</li>
        <li>This notebook uses text based models along with their versions that were available at the time of writing. Update these as required.</li>
        <li>At the time of writing this notebook, Amazon Bedrock was only available in <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/bedrock-regions.html">these supported AWS Regions</a>. If you are running this notebook from any other AWS Region, then you have to change the Amazon Bedrock client's region and/or endpoint URL parameters to one of those supported AWS Regions that has Anthropic Claude 3 and Titan Text Embeddings V2 . Follow the guidance in the <i>Organize imports</i> section of this notebook.</li>
        <li>This notebook is recommended to be run with a minimum instance size of <i>ml.t3.medium</i> and
            <ul>
                <li>With <i>Amazon Linux 2, Jupyter Lab 3</i> as the platform identifier on an Amazon SageMaker Notebook instance.</li>
                <li> (or)
                <li>With <i>Data Science 3.0</i> as the image on an Amazon SageMaker Studio Notebook.</li>
            <ul>
        </li>
        <li>At the time of this writing, the most relevant latest version of the Kernel for running this notebook,
            <ul>
                <li>On an Amazon SageMaker Notebook instance was <i>conda_python3</i></li>
                <li>On an Amazon SageMaker Studio Notebook was <i>Python 3</i></li>
            </ul>
        </li>
    </ul>
</div>

**Table of Contents:**

1. [Complete prerequisites](#Complete%20prerequisites)

    a. [Check and configure access to the Internet](#Check%20and%20configure%20access%20to%20the%20Internet)

    b. [Install required software libraries](#Install%20required%20software%20libraries)
    
    c. [Configure logging](#Configure%20logging)
    
    d. [Organize imports](#Organize%20imports)
    
    e. [Set AWS Region and boto3 config](#Set%20AWS%20Region%20and%20boto3%20config)
    
    f. [Enable model access in Amazon Bedrock](#Enable%20model%20access%20in%20Amazon%20Bedrock)
    
    g. [Check and configure security permissions](#Check%20and%20configure%20security%20permissions)
    
    h. [Create common objects](#Create%20common%20objects)
    
    i. [Get details of Knowledge Bases](#Get%20details%20of%20Knowledge%20Bases)

 2. [Load data to Knowledge Bases](#Load%20data%20to%20Knowledge%20Bases)
    
    a. [Step0a: Copy provided data to S3](#Load%20to%20KB%20Step0b)
    
    b. [Steps 0b to 0e: Sync to Knowledge Base](#Load%20to%20KB%20Steps0c%20to%200e)
 
 3. [Process query](#Process%20query)
 
     a. [Step 1: User query and non-KB query](#User%20query)
     
     b. [Steps 2a and 2b: Determine the KB id](#Determine%20the%20KB%20id)
     
     c. [Steps 3a through 5: Retrieve and generate](#Retrieve%20and%20generate)
 
 4. [Conclusion](#Conclusion)
 
 5. [Frequently Asked Questions (FAQs)](#FAQs)

##  1. Complete prerequisites <a id ='Complete%20prerequisites'> </a>

Check and complete the prerequisites.

###  A. Check and configure access to the Internet <a id ='Check%20and%20configure%20access%20to%20the%20Internet'> </a>
This notebook requires outbound access to the Internet to download the required software updates and to download the dataset.  You can either provide direct Internet access (default) or provide Internet access through an [Amazon VPC](https://aws.amazon.com/vpc/).  For more information on this, refer [here](https://docs.aws.amazon.com/sagemaker/latest/dg/appendix-notebook-and-internet-access.html).

### B. Install required software libraries <a id ='Install%20required%20software%20libraries'> </a>
This notebook requires the following libraries:
* [SageMaker Python SDK version 2.x](https://sagemaker.readthedocs.io/en/stable/v2.html)
* [Python 3.10.x](https://www.python.org/downloads/release/python-3100/)
* [Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html)
* [LangChain](https://www.langchain.com/)

Run the following cell to install the required libraries.

<div class="alert alert-block alert-warning">  
    <b>Note:</b> At the end of the installation, the Kernel will be forcefully restarted immediately. Please wait 10 seconds for the kernel to come back before running the next cell.
</div>

In [ ]:
!pip install boto3==1.35.43
!pip install langchain==0.3.4
!pip install langchain-aws==0.2.2
!pip install langchain-community==0.3.3
!pip install requests==2.32.3
!pip install sagemaker==2.232.2
!pip install ragas


import IPython

IPython.Application.instance().kernel.do_shutdown(True)

### C. Configure logging <a id ='Configure%20logging'> </a>

####  a. System logs (Optional) <a id='Configure%20system%20logs%20(Optional)'></a>

System logs refers to the logs generated by the notebook's interactions with the underlying notebook instance. Some examples of these are the logs generated when loading or saving the notebook.

These logs are automatically setup when the notebook instance is launched.

These logs can be accessed through the [Amazon CloudWatch Logs](https://docs.aws.amazon.com/AmazonCloudWatch/latest/logs/WhatIsCloudWatchLogs.html) console in the same AWS Region where this notebook is running.
* When running this notebook in an Amazon SageMaker Notebook instance, navigate to the following location,
    * <i>CloudWatch > Log groups > /aws/sagemaker/NotebookInstances > {notebook-instance-name}/jupyter.log</i>
* When running this notebook in an Amazon SageMaker Studio Notebook, navigate to the following locations,
    * <i>CloudWatch > Log groups > /aws/sagemaker/studio > {sagmaker-domain-name}/{user-name}/KernelGateway/{notebook-instance-name}</i>
    * <i>CloudWatch > Log groups > /aws/sagemaker/studio > {sagmaker-domain-name}/{user-name}/JupyterServer/default</i>

If you want to find out the name of the underlying instance where this notebook is running, uncomment the following code cell and run it.

In [ ]:
'''
import json

notebook_name = ''
resource_metadata_path = '/opt/ml/metadata/resource-metadata.json'
with open(resource_metadata_path, 'r') as metadata:
    notebook_name = (json.load(metadata))['ResourceName']
print("Notebook instance name: '{}'".format(notebook_name))
'''

####  b. Application logs <a id='Configure%20application%20logs'></a>

Application logs refers to the logs generated by running the various code cells in this notebook. To set this up, instantiate the [Python logging service](https://docs.python.org/3/library/logging.html) by running the following cell. You can configure the default log level and format as required.

By default, this notebook will only print the logs to the corresponding cell's output console.

In [6]:
import logging
import os

# Set the logging level and format
log_level = logging.INFO
log_format = '%(asctime)s - %(levelname)s - %(message)s'
logging.basicConfig(level=log_level, format=log_format)

# Save these in the environment variables for use in the helper scripts
os.environ['LOG_LEVEL'] = str(log_level)
os.environ['LOG_FORMAT'] = log_format

###  D. Organize imports <a id ='Organize%20imports'> </a>

Organize all the library and module imports for later use.

In [7]:
import boto3
import langchain
import sagemaker
import sys
import time
import pprint as pp
from botocore.config import Config

# Import the helper functions from the 'scripts' folder
sys.path.append(os.path.join(os.getcwd(), "scripts"))
#logging.info("Updated sys.path: {}".format(sys.path))
from helper_functions import *

Print the installed versions of some of the important libraries.

In [ ]:
logging.info("Python version : {}".format(sys.version))
logging.info("Boto3 version : {}".format(boto3.__version__))
logging.info("SageMaker Python SDK version : {}".format(sagemaker.__version__))
logging.info("LangChain version : {}".format(langchain.__version__))
logging.info("Requests version : {}".format(requests.__version__))

###  E. Set AWS Region and boto3 config <a id ='Set%20AWS%20Region%20and%20boto3%20config'> </a>

Get the current AWS Region (where this notebook is running) and the SageMaker Session. These will be used to initialize some of the clients to AWS services using the boto3 APIs.

<div class="alert alert-block alert-info">  
<b>Note:</b> All the AWS services used by this notebook except Amazon Bedrock will use the current AWS Region. For Bedrock, follow the guidance in the next cell.
</div>

<div class="alert alert-block alert-warning">  
<b>Note:</b> At the time of writing this notebook, Amazon Bedrock was only available in <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/bedrock-regions.html">these supported AWS Regions</a>. If you are running this notebook from any other AWS Region, then you have to change the Amazon Bedrock client's region and/or endpoint URL parameters to one of those supported AWS Regions that has Anthropic Claude 3 and Titan Text Embeddings V2 . In order to do this, this notebook will use the value specified in the environment variable named <mark>AMAZON_BEDROCK_REGION</mark>. If this is not specified, then the notebook will default to <mark>us-west-2 (Oregon)</mark> for Amazon Bedrock.
</div>



In [ ]:
# Get the AWS Region, SageMaker Session and IAM Role references
my_session = boto3.session.Session()
logging.info("SageMaker Session: {}".format(my_session))
my_iam_role = sagemaker.get_execution_role()
logging.info("Notebook IAM Role: {}".format(my_iam_role))
my_region = my_session.region_name
logging.info("Current AWS Region: {}".format(my_region))

# Explicity set the AWS Region for Amazon Bedrock clients
AMAZON_BEDROCK_DEFAULT_REGION = "us-west-2"
br_region = os.environ.get('AMAZON_BEDROCK_REGION')
if br_region is None:
    br_region = AMAZON_BEDROCK_DEFAULT_REGION
elif len(br_region) == 0:
    br_region = AMAZON_BEDROCK_DEFAULT_REGION
logging.info("AWS Region for Amazon Bedrock: {}".format(br_region))

Set the timeout and retry configurations that will be applied to all the boto3 clients used in this notebook.

In [14]:
# Increase the standard time out limits in the boto3 client from 1 minute to 3 minutes
# and set the retry limits
my_boto3_config = Config(
    connect_timeout = (60 * 3),
    read_timeout = (60 * 3),
    retries = {
        'max_attempts': 10,
        'mode': 'standard'
    }
)

###  F. Enable model access in Amazon Bedrock <a id ='Enable%20model%20access%20in%20Amazon%20Bedrock'> </a>

<div class="alert alert-block alert-danger">
    <b>Note:</b> Before proceeding further with this notebook, you must enable access to the 'Anthropic Claude 3' and 'Titan Text Embeddings V2' models on Amazon Bedrock by following the instructions <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html">here</a>. You need to submit the use case details. Otherwise, you will get an authorization error.
</div>

<div class="alert alert-block alert-warning">  
<b>Note:</b> You will have to do this manually after reading the End User License Agreement (EULA) for each of the models that you want to enable. Unless you explicitly disable it, this is a one-time setup for each model in an AWS account.
</div>

Run the following cell to print the Amazon Bedrock model access page URL for the AWS Region that was selected earlier.

In [ ]:
# Print the Amazon Bedrock model access page URL
logging.info("Amazon Bedrock model access page - https://{}.console.aws.amazon.com/bedrock/home?region={}#/modelaccess"
             .format(br_region, br_region))

<div class="alert alert-block alert-info">  
<b>Note:</b> For running this notebook, you need access to only the Anthropic Claude 3 Haiku and Sonnet models, and the Titan Text Embeddings V2  model. 
</div>

###  G. Check and configure security permissions <a id ='Check%20and%20configure%20security%20permissions'> </a>
This notebook uses the IAM role attached to the underlying notebook instance.  To view the name of this role, run the following cell. This IAM role should have the following permissions,
1. Full access to invoke Large Language Models (LLMs) on Amazon Bedrock.
2. Full access to use Knowledge Bases for Amazon Bedrock.
3. Full access to read and write to the Amazon OpenSearch Serverless collection associated with the Knowledge Base.
4. Full access to read and write to the Amazon S3 bucket associated with the Knowledge Base.
5. Access to write to Amazon CloudWatch Logs.

<div class="alert alert-block alert-info">
<b>Note:</b>  If you are running this notebook as part of a workshop session, by default, all these permissions will be setup.
</div>

Run the following cell to print the details of the IAM role attached to the underlying notebook instance.

In [ ]:
# Print the IAM role ARN and console URL
logging.info("This notebook's IAM role is '{}'".format(my_iam_role))
arn_parts = my_iam_role.split('/')
logging.info("Details of this IAM role are available at https://{}.console.aws.amazon.com/iamv2/home?region={}#/roles/details/{}?section=permissions"
             .format(my_region, my_region, arn_parts[len(arn_parts) - 1]))

###  H. Create common objects <a id='Create%20common%20objects'></a>

To begin, let's create the boto3 clients.

In [ ]:
# Create the Amazon S3 client
s3_client = boto3.client("s3", region_name = br_region, config = my_boto3_config)

# Create the Amazon OpenSearch Serverless client
aoss_client = boto3.client("opensearchserverless", region_name = br_region, config = my_boto3_config)

# Create the Amazon Bedrock client
bedrock_client = boto3.client("bedrock", region_name = br_region, endpoint_url = "https://bedrock.{}.amazonaws.com"
                              .format(br_region), config = my_boto3_config)

# Create the Amazon Bedrock runtime client
bedrock_rt_client = boto3.client("bedrock-runtime", region_name = br_region, config = my_boto3_config)

# Create the Agents for Amazon Bedrock client
bedrock_agt_client = boto3.client("bedrock-agent", region_name = br_region, config = my_boto3_config)

# Create the Agents for Amazon Bedrock runtime client
bedrock_agt_rt_client = boto3.client("bedrock-agent-runtime", region_name = br_region, config = my_boto3_config)

List all Anthropic Claude 3 LLMs on Amazon Bedrock that are offered through the On-Demand throughput pricing model. This will help you pick the model-ids that you will use further down in this notebook.

For more information on this, refer [here](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html#model-ids-arns).

In [ ]:
# Note: 'print_claude_3_llm_info' available through ./scripts/helper_functions.py
print_claude_3_llm_info(bedrock_client, 'ON_DEMAND')

Create the common objects.

In [ ]:
# Specify the Anthropic Claude 3 model-id
#model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
model_id = "anthropic.claude-3-haiku-20240307-v1:0"

# Specify the URL of the data to be used for RAG
kb_1_category = "Amazon Bedrock"
kb_2_category = "Code"

# Specify the Amazon S3 bucket key prefix
# Note: The bucket name will be automatically determined
# from the Knowledge Base data source when executing cells
# further down in this notebook
s3_key_prefix = "rag_data"

# Specify the name and location of the prompt templates
prompt_templates_dir = os.path.join(os.getcwd(), "prompt_templates")
kb_router_system_prompt_template = 'kb_router_system_prompt_template.txt'
kb_router_user_prompt_template = 'kb_router_user_prompt_template.txt'
final_system_prompt_template = 'final_system_prompt_template.txt'
final_user_prompt_template = 'final_user_prompt_template.txt'

# Specify and create the required output directories
kb_1_rag_data_dir = os.path.join(os.getcwd(), "kb_1_rag_data")
kb_2_rag_data_dir = os.path.join(os.getcwd(), "kb_2_rag_data")
os.makedirs(kb_1_rag_data_dir, exist_ok = True)
os.makedirs(kb_2_rag_data_dir, exist_ok = True)

# Specify the maximum number of query results to retrieve from the KBs
max_query_results = 10

###  I. Get details of Knowledge Bases <a id='Get%20details%20of%20Knowledge%20Bases'></a>

<div class="alert alert-block alert-info">
<b>Note:</b> For the purpose of running this notebook, two new Knowledge Bases (KBs) must be created in the same AWS Region as Amazon Bedrock that was configured in Step 1E of this notebook.
<p>These KBs must meet the following requirements:
    <ul>
        <li>Must be in 'ACTIVE' status.</li>
        <li>Must have an Amazon OpenSearch Serverless collection as the vector index.</li>
        <li>Must have an Amazon S3 bucket as the data source.</li>
        <li>Data source must be in 'AVAILABLE' status.</li>
        <li>Embeddings model must be 'Titan Text Embeddings V2 '.</li>
    </ul>
</p>
<p>If you are running this notebook as part of a workshop session, by default, two KBs that meet all these requirements will be pre-created and ready to use.</p>
</div>

<div class="alert alert-block alert-danger">
    <b>Note:</b> If you are running this notebook outside of a workshop session, then, you must create two KBs as specified above. Otherwise, this notebook will fail. You can follow the procedure described <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base-create.html">here</a> to create a KB.
</div>

Specify the names of the KBs to be used in this notebook by running the following cell.

In [ ]:
kb_1_name = 'rag-router-kb-1'
kb_2_name = 'rag-router-kb-2'

Run the following cell to verify if the specified KBs exist and if they meet all the requirements. In the process, retreive the S3 bucket names that are set as the data sources for these KBs.

In [ ]:
# Note: 'get_kb_details' available through ./scripts/helper_functions.py
kb_1_id, kb_1_ds_id, kb_1_s3_bucket_name, kb_1_aoss_collection_arn = get_kb_details(bedrock_agt_client, kb_1_name)
kb_2_id, kb_2_ds_id, kb_2_s3_bucket_name, kb_2_aoss_collection_arn = get_kb_details(bedrock_agt_client, kb_2_name)

## 2. Load data to Knowledge Bases <a id ='Load%20data%20to%20Knowledge%20Bases'> </a>

![](./images/load_to_KBs.png)

###  B. Step 0a: Copy provided data to S3 <a id='Load%20to%20KB%20Step0b'></a>

Copy the provided files to the corresponding Amazon S3 buckets. These buckets will be the [data sources](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base-ds.html) for the Knowledge Bases (KBs).

In [ ]:
# Note: 'upload_to_s3' available through ./scripts/helper_functions.py
upload_to_s3("pdf_docs", kb_1_s3_bucket_name, s3_key_prefix)
upload_to_s3("blogs", kb_2_s3_bucket_name, s3_key_prefix)

###  C. Step 0c Create web data sources<a id='Load%20to%20KB%20Steps0c%20to%200e'></a>

Trigger the [sync operation](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base-ingest.html) on the Knowledge Bases to load the files from that data sources (S3 buckets) to the [vector index](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base-setup.html) configured for in those Knowledge Bases (KBs). In the process of loading, the data will be chunked and converted to vectors using the Embeddings Model specified for those KBs. 

We are going to create two separate data sources. One with fixed size chunking and one with hierarchical chunking. Hierarchical chunking organizes chunks into parent and child chunks that aligns with the structure of your input document. We will use this for the Bedrock user guide, which we are indexing. To learn more about chunking, look at our documentation [here](https://docs.aws.amazon.com/bedrock/latest/userguide/kb-chunking-parsing.html)

In [ ]:
data_sources_kb1 = [
{"type": "S3", "bucket_name": kb_1_s3_bucket_name, "chunking_strategy":"HIERARCHICAL"}, 
]
data_sources_kb2 = [
{"type": "S3", "bucket_name": kb_2_s3_bucket_name, "chunking_strategy":"FIXED_SIZE"},
]
# to create a semantic chunking db, use the following:
# {"type": "S3", "bucket_name": kb_3_s3_bucket_name, "chunking_strategy":"SEMANTIC"},

In [ ]:
# Function to create KB
def create_ds(data_sources, kb_id, name = "web-data-source", description = "aws blog web data source"):
    ds_list=[]
    for idx, ds in enumerate(data_sources):
        # Ingest strategy - How to ingest data from the data source
        if ds['chunking_strategy'] == "FIXED_SIZE":
            chunkingStrategyConfiguration = {
                "chunkingStrategy": "FIXED_SIZE", 
                "fixedSizeChunkingConfiguration": {
                    "maxTokens": 512,
                    "overlapPercentage": 20
                }
            }
        elif ds['chunking_strategy'] == "HIERARCHICAL":
            chunkingStrategyConfiguration = {
                "chunkingStrategy": "HIERARCHICAL", 
                "hierarchicalChunkingConfiguration": {
                    "overlapTokens": 100,
                    "levelConfigurations":[
                    {
                        "maxTokens":512
                    },
                    {
                        "maxTokens":256
                    }]
                    
                }
            }
        elif ds['chunking_strategy'] == "SEMANTIC":
            chunkingStrategyConfiguration = {
                "chunkingStrategy": "SEMANTIC", 
            }
        
        # The data source to ingest documents from, into the OpenSearch serverless knowledge base index
        
        s3DataSourceConfiguration = {
                "type": "S3",
                "s3Configuration":{
                    "bucketArn": "",
                    # "inclusionPrefixes":["*.*"] # you can use this if you want to create a KB using data within s3 prefixes.
                    }
            }
        

        webcrawlerDataSourceConfiguration = {
            "webConfiguration": {
                "sourceConfiguration": {
                    "urlConfiguration": {
                        "seedUrls": []
                    }
                },
                "crawlerConfiguration": {
                    "crawlerLimits": {
                        "rateLimit": 50
                    },
                    "scope": "HOST_ONLY",
                    "inclusionFilters": [],
                    "exclusionFilters": []
                }
            },
            "type": "WEB"
        }

        # Set the data source configuration based on the Data source type

        if ds['type'] == "S3":
            print(f'{idx +1 } data source: S3')
            ds_name = f'{name}-{ds["bucket_name"]}'
            s3DataSourceConfiguration["s3Configuration"]["bucketArn"] = f'arn:aws:s3:::{ds["bucket_name"]}'
            # print(s3DataSourceConfiguration)
            data_source_configuration = s3DataSourceConfiguration
        
        if ds['type'] == "WEB":
            print(f'{idx +1 } data source: WEB')
            ds_name = f'{name}-web'
            webcrawlerDataSourceConfiguration['webConfiguration']['sourceConfiguration']['urlConfiguration']['seedUrls'] = ds['seedUrls']
            webcrawlerDataSourceConfiguration['webConfiguration']['crawlerConfiguration']['inclusionFilters'] = ds['inclusionFilters']
            webcrawlerDataSourceConfiguration['webConfiguration']['crawlerConfiguration']['exclusionFilters'] = ds['exclusionFilters']
            # print(webcrawlerDataSourceConfiguration)
            data_source_configuration = webcrawlerDataSourceConfiguration
            

        # Create a DataSource in KnowledgeBase 
        create_ds_response = bedrock_agt_client.create_data_source(
            name = ds_name,
            description = description,
            knowledgeBaseId = kb_id,
            dataSourceConfiguration = data_source_configuration,
            vectorIngestionConfiguration = {
                "chunkingConfiguration": chunkingStrategyConfiguration
            }
        )
        ds = create_ds_response["dataSource"]
        pp.pprint(ds)
        ds_list.append(ds)
    return ds_list

Create our data sources

In [ ]:
ds_list1 = create_ds(data_sources_kb1, kb_1_id, name = "bedrock-user-guide", description = "Amazon Bedrock user guide")
ds_list2 = create_ds(data_sources_kb2, kb_2_id, name = "aws-blogs", description = "aws blog data source")

###  D. Step 0d: Sync to Knowledge Base <a id='Load%20to%20KB%20Step0b'></a>

Let's trigger our sync operations. We will check the status of the ingestion every 5 seconds until failed or complete.

In [ ]:
# Note: 'sync_to_kb' available through ./scripts/helper_functions.py
sync_to_kb(bedrock_agt_client, ds_list1[0]['dataSourceId'], kb_1_id, 'Sync PDF from S3 to vector index')
sync_to_kb(bedrock_agt_client,  ds_list2[0]['dataSourceId'], kb_2_id, 'sync AWS blogs to vector index')

<div class="alert alert-block alert-info">
    <b>Note:</b> Even if the sync operation is in 'COMPLETE' status, it may take up to 60 seconds for the data to be available for reading from the vector index.
</div>

## 3. Process query <a id ='Process%20query'> </a>

Based on the specified query, determine the right Knowledge Base (KB) to retrieve from, perform the retrieval and generate a response.

![](./images/retrieve_and_generate.png)

###  A. Step 1a and 1b: User query and non-KB response <a id='User%20query'></a>

Let's set the user query and try to answer it without a knowledge base.

In [23]:
# Category: Code Samples
#query = 'Get me code to create a new KB'

# Category: Amazon Bedrock
query = 'What is a Bedrock KB?'

Let's try to answer this question without our knowledge base. Here we will use the [Amazon Bedrock Converse API](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html#message-inference-examples). The converse API provides an easy way for us to converse with LLMs hosted in Bedrock. Let's ask Claude Sonnet about our question.

To learn more about prompting guidelines, you can find information [here](https://docs.aws.amazon.com/bedrock/latest/userguide/general-guidelines-for-bedrock-users.html#:~:text=Temperature%20is%20a%20value%20between,from%20LLMs%20on%20Amazon%20Bedrock.)

In [ ]:
# Temperature is a value between 0 and 1, it regulates the creativity of LLMs' responses.
temperature = 0
# Top-k sampling refines the selection process by limiting the pool of potential next tokens to the top-k most likely candidates.
top_k = 200

# Base inference parameters to use.
inference_config = {"temperature": temperature}
# Additional inference parameters to use.
additional_model_fields = {"top_k": top_k}
messages = [{
    "role": "user",
    "content": [{"text": query}]
}]
# Setup the system prompts and messages to send to the model. System prompts can be used to give your model a specific persona or set other context.
system_prompts = [{"text": "You are an aws expert"}]

response = bedrock_rt_client.converse(
        modelId=model_id,
        messages=messages,
        system=system_prompts,
        inferenceConfig=inference_config,
        additionalModelRequestFields=additional_model_fields
    )
logging.info(response["output"]["message"]["content"][0]["text"])

Let's try again, but this time using our knowledge base. 

###  B. Steps 2a and 2b: Determine the KB id <a id='Determine%20the%20KB%20id'></a>

Determine the Knowledge Base (KB) id to retrieve from by prompting the LLM with the query and the list of categories and KB ids.

Let's take a look at the prompt we are using. The basics structure is
- we specify the available knowledge bases and the category they are associated with
- the output format we are looking for
- explicit instructions on how to evaluate the user query.

Our output from the model then is a specific KB ID we can then query to get context for our next LLM call.

In [ ]:
# get our prompt and print it out.
router_prompt_template_file_path = os.path.join(prompt_templates_dir, kb_router_user_prompt_template)
router_prompt_template = PromptTemplate.from_file(router_prompt_template_file_path)
logging.info(router_prompt_template.template)

In [ ]:
# Invoke the LLM to determin the KB id
temperature = 0

# Base inference parameters to use.
inference_config = {"temperature": temperature}

# filled prompt template
filled_router_template = router_prompt_template.format(QUERY=query, 
                                         CATEGORY_1="code", 
                                         CATEGORY_2="docs")

router_messages = [{
    "role": "user",
    "content": [{"text": filled_router_template}]
}]
# Setup the system prompts and messages to send to the model. System prompts can be used to give your model a specific persona or set other context.
system_prompts = [{"text": "You are an aws expert"}]

response = bedrock_rt_client.converse(
        modelId=model_id,
        messages=router_messages,
        system=system_prompts,
        inferenceConfig=inference_config,
        additionalModelRequestFields=additional_model_fields
    )
query_category = response["output"]["message"]["content"][0]["text"]

#small dictionary mapping kb ids to the categories 
category_dict = {"code":kb_1_id,
 "docs":kb_2_id}

query_kb_id = category_dict[query_category]
logging.info(query_category)

# Process the response
if query_category == 'UNKNOWN':
    logging.error("The LLM was unable to identify the category of the specified query from the list of provided categories.")
else:
    logging.info("The LLM was able to identify the category of the specified query from the list of provided categories.")

In [ ]:
# Print the output
logging.info("\n\nQUERY: {}\n\nCATEGORY: {}".format(query, query_category))

###  C. Steps 3a through 5: Retrieve and generate <a id='Retrieve%20and%20generate'></a>

Perform a semantic search on the vector store in the Knowledge Base (KB) for the user query and retrieve the results. The following cell shows you two ways to do this - one using the LangChain API and the other using the boto3 API.

In [ ]:

# Use the LangChain API
# Note: 'retrieve_from_kb_using_lc' available through ./scripts/helper_functions.py
query_lookup_results = retrieve_from_kb_using_lc(bedrock_agt_rt_client, query_kb_id, query, max_query_results)

# Use the boto3 API
# Note: 'retrieve_from_kb_using_boto3' available through ./scripts/helper_functions.py
#query_lookup_results = retrieve_from_kb_using_boto3(bedrock_agt_rt_client, query_kb_id, query, max_query_results)

# Print the retrieval count
logging.info("Retrieved {} result(s) for the specified query '{}' from the Knowledge Base '{}'.".format(len(query_lookup_results),
                                                                                                        query,
                                                                                                        query_kb_id))

Construct the prompt, invoke the LLM and generate the response.

In [ ]:
# get our prompt and print it out.
final_prompt_template_file_path = os.path.join(prompt_templates_dir, final_user_prompt_template)
final_prompt_template = PromptTemplate.from_file(final_prompt_template_file_path)
logging.info(final_prompt_template.template)

In [ ]:
# Invoke the LLM to determine the KB id
temperature = 0

# Base inference parameters to use.
inference_config = {"temperature": temperature}

# filled prompt template
filled_final_template = final_prompt_template.format(QUERY=query, 
                                         CONTEXT=query_lookup_results, 
                                        )

final_messages = [{
    "role": "user",
    "content": [{"text": filled_final_template}]
}]
# Setup the system prompts and messages to send to the model. System prompts can be used to give your model a specific persona or set other context.
system_prompts = [{"text": "You are an aws expert"}]

response = bedrock_rt_client.converse(
        modelId=model_id,
        messages=final_messages,
        system=system_prompts,
        inferenceConfig=inference_config,
        additionalModelRequestFields=additional_model_fields
    )
query_response = response["output"]["message"]["content"][0]["text"]

logging.info("\n\nQUERY: {}\n\nRESPONSE: {}".format(query, query_response))

Let's try another query, but this time we will ask a question that will be routed to a different KB. Let's run it all together.

In [ ]:
# Category: Large Language Models
query = 'How can I create a new knowledge base using boto3?'

# filled prompt template
filled_router_template = router_prompt_template.format(QUERY=query, 
                                         CATEGORY_1="code", 
                                         CATEGORY_2="docs")

router_messages = [{
    "role": "user",
    "content": [{"text": filled_router_template}]
}]
# Setup the system prompts and messages to send to the model. System prompts can be used to give your model a specific persona or set other context.
system_prompts = [{"text": "You are an aws expert"}]

# run router LLM call
response = bedrock_rt_client.converse(
        modelId=model_id,
        messages=router_messages,
        system=system_prompts,
        inferenceConfig=inference_config,
        additionalModelRequestFields=additional_model_fields
    )
query_category = response["output"]["message"]["content"][0]["text"]

query_kb_id = category_dict[query_category]
logging.info(query_category)

# retrieve results from our knowledge base
query_lookup_results = retrieve_from_kb_using_lc(bedrock_agt_rt_client, query_kb_id, query, max_query_results)

# filled prompt template
filled_final_template = final_prompt_template.format(QUERY=query, 
                                         CONTEXT=query_lookup_results, 
                                        )

final_messages = [{
    "role": "user",
    "content": [{"text": filled_final_template}]
}]

# execute final prompt
response = bedrock_rt_client.converse(
        modelId=model_id,
        messages=final_messages,
        system=system_prompts,
        inferenceConfig=inference_config,
        additionalModelRequestFields=additional_model_fields
    )
query_response = response["output"]["message"]["content"][0]["text"]

logging.info("\n\nQUERY: {}\n\nRESPONSE: {}".format(query, query_response))


## 4. Conclusion <a id='Conclusion'></a>

We have now seen how to build an advanced RAG router based assistant with Amazon Bedrock. Now you can move on to the next notebook `prompt_flow_generation_aws.ipynb` to see how to create a prompt flow application for your chat assistant!

## 5. Frequently Asked Questions (FAQs) <a id='FAQs'></a>

**Q: What AWS services are used in this notebook?**

Amazon Bedrock, Amazon OpenSearch Serverless, Amazon S3, AWS Identity and Access Management (IAM), Amazon CloudWatch, and Amazon SageMaker Notebook instance (or) Amazon SageMaker Studio Notebook depending on what you use to run the notebook.

**Q: Will Amazon Bedrock capture and store my data?**

Amazon Bedrock doesn't use your prompts and continuations to train any AWS models or distribute them to third parties. Your training data isn't used to train the base Amazon Titan models or distributed to third parties. Other usage data, such as usage timestamps, logged account IDs, and other information logged by the service, is also not used to train the models.

Amazon Bedrock uses the fine tuning data you provide only for fine tuning an Amazon Titan model. Amazon Bedrock doesn't use fine tuning data for any other purpose, such as training base foundation models.

Each model provider has an escrow account that they upload their models to. The Amazon Bedrock inference account has permissions to call these models, but the escrow accounts themselves don't have outbound permissions to Amazon Bedrock accounts. Additionally, model providers don't have access to Amazon Bedrock logs or access to customer prompts and continuations.

Amazon Bedrock doesn’t store or log your data in its service logs.

**Q: What models are supported by Amazon Bedrock?**

Go [here](https://docs.aws.amazon.com/bedrock/latest/userguide/models-supported.html).

**Q: What is the difference between On-demand and Provisioned Throughput in Amazon Bedrock?**

With the On-Demand mode, you only pay for what you use, with no time-based term commitments. For text generation models, you are charged for every input token processed and every output token generated. For embeddings models, you are charged for every input token processed. A token is comprised of a few characters and refers to the basic unit that a model learns to understand user input and prompt to generate results. For image generation models, you are charged for every image generated.

With the Provisioned Throughput mode, you can purchase model units for a specific base or custom model. The Provisioned Throughput mode is primarily designed for large consistent inference workloads that need guaranteed throughput. Custom models can only be accessed using Provisioned Throughput. A model unit provides a certain throughput, which is measured by the maximum number of input or output tokens processed per minute. With this Provisioned Throughput pricing, charged by the hour, you have the flexibility to choose between 1-month or 6-month commitment terms.

**Q: Where can I find customer references for Amazon Bedrock?**

Go [here](https://aws.amazon.com/bedrock/testimonials/).

**Q: Where can I find resources for prompt engineering?**

[Prompt Engineering Guide](https://www.promptingguide.ai/).

**Q: Where can learn more about Corrective RAG?**

Go [here](https://arxiv.org/abs/2401.15884).

**Q: Is LangChain mandatory to use Amazon Bedrock?**

No. You can interact with Amazon Bedrock using the [Bedrock API](https://docs.aws.amazon.com/bedrock/latest/APIReference/welcome.html) or language-specific [AWS SDKs](https://aws.amazon.com/developer/tools/). 

**Q: How do I get started with LangChain?**

Go [here](https://python.langchain.com/docs/get_started/introduction).

**Q: Where can I find pricing information for the AWS services used in this notebook?**

- Amazon Bedrock pricing - go [here](https://aws.amazon.com/bedrock/pricing/).
- Amazon OpenSearch Serverless pricing - go [here](https://aws.amazon.com/opensearch-service/pricing/) and navigate to the <i>Serverless</i> section.
- Amazon S3 pricing - go [here](https://aws.amazon.com/s3/pricing/).
- AWS Identity and Access Management (IAM) pricing - free.
- Amazon CloudWatch pricing - go [here](https://aws.amazon.com/cloudwatch/pricing/).
- Amazon SageMaker Notebook instance (or) Amazon SageMaker Studio Notebook pricing - go [here](https://aws.amazon.com/sagemaker/pricing/).